In [ ]:
import concurrent.futures
import datetime
import json
import requests
from requests.cookies import RequestsCookieJar
from dataclasses import dataclass


@dataclass
class ServerInfo:
    name: str
    base_url: str
    query_override: str = None
    auth_cookie: RequestsCookieJar = None


class DataRetriever:
    secrets_scope = "NMIS"
    api_id_secret_key = "nmis_api_id"
    api_pwd_secret_key = "nmis_api_pwd"
    auth_endpoint = "/omk/opCharts/login?username={api_id}&password={api_pwd}"
    data_endpoint = "/en/omk/opCharts/v1/nodes"
    single_node_query = 'properties=["info."]'
    errors: list[Exception]

    def __init__(
        self,
        servers: list[ServerInfo],
        target_dir: str,
        default_query: str,
        dbutils,
        should_process_individual_node_ids: bool = False,
        individual_target_dir: str = None,
    ):
        self.errors = []
        self.servers = servers
        self.target_dir = target_dir
        self.default_query = default_query
        self.dbutils = dbutils
        self.should_process_individual_node_ids = should_process_individual_node_ids
        self.individual_target_dir = individual_target_dir
        self.api_id = self.dbutils.secrets.get(
            self.secrets_scope, self.api_id_secret_key
        )
        self.api_pwd = self.dbutils.secrets.get(
            self.secrets_scope, self.api_pwd_secret_key
        )

    def get_auth_cookie(self, server_url) -> RequestsCookieJar:
        url = f"{server_url}{self.auth_endpoint}".format(
            api_id=self.api_id, api_pwd=self.api_pwd
        )
        auth_response = requests.post(url)
        if auth_response.status_code == 200:
            return auth_response.cookies
        raise Exception(
            f"Auth call failed for url: {server_url}. Response:{auth_response}"
        )

    @staticmethod
    def make_api_call(api_url: str, cookies: RequestsCookieJar) -> any:
        response = requests.get(api_url, cookies=cookies)

        if response.status_code == 200:
            return response.json()
        raise Exception(f"Data call failed for url: {api_url}. Response:{response}")

    @staticmethod
    def now() -> datetime:
        return datetime.datetime.now()

    def write_data(self, target_dir, server_name, data):
        currentDatetime = DataRetriever.now().strftime("%Y%m%d%H%M%S")
        target_filename = f"{target_dir}/{server_name}-{currentDatetime}.json"
        self.dbutils.fs.put(target_filename, json.dumps(data, ensure_ascii=False))

    def send_teams_notification(self, error_message):
        secrets_scope = "NMIS"
        teams_notification_key = "nmis_teams_notification"
        teams_webhook_url = dbutils.secrets.get(secrets_scope, teams_notification_key)

        # Convert non-string error instances to strings
        error_messages = [
            str(error) for error in self.errors if isinstance(error, Exception)
        ]

        # Add the specific error message passed to the method
        error_messages.append(error_message)

        # Join all error messages with a newline character
        formatted_errors = "\n".join(error_messages)

        # Escape backslashes by doubling them within the f-string
        message = {"text": f"NMIS8 - Error occurred in script:\n\n{formatted_errors}"}
        response = requests.post(teams_webhook_url, json=message)
        if response.status_code != 200:
            raise Exception(
                f"Failed to send Teams notification. Status code: {response.status_code}"
            )

    def process_all_servers(self):
        self.dbutils.fs.mkdirs(self.target_dir)
        for server in self.servers:
            query = server.query_override or self.default_query
            data_url = f"{server.base_url}{self.data_endpoint}?{query}"
            try:
                server.auth_cookie = self.get_auth_cookie(server.base_url)
                response_data = self.make_api_call(data_url, server.auth_cookie)
                self.write_data(self.target_dir, server.name, response_data)
            except Exception as e:
                error_message = f"Error occurred during server {server.name}"
                self.errors.append(error_message)

        if self.should_process_individual_node_ids:
            self.process_individual_node_ids()

        if self.errors:
            self.send_teams_notification("\n\n".join(self.errors))

    """
    Reads all json files in the read_dir. They are expected to be json arrays of node_ids.
    Calls the API for node info for each node ID found, then dumps full node JSON info to a
    file in the target_dir for each file that was in the read_dir.
    Deletes read_dir file after processing.
    """

    def process_individual_node_ids(self):
        read_dir = self.target_dir
        self.dbutils.fs.mkdirs(self.individual_target_dir)

        for file in self.dbutils.fs.ls(read_dir):
            print(f"Processing file: {file.path}")
            filestring = self.dbutils.fs.head(file.path, 200000)
            node_ids = json.loads(filestring)

            server = next(
                filter(
                    lambda server_info: file.name.startswith(server_info.name),
                    self.servers,
                ),
                None,
            )
            print(f"Using server info {server}")

            node_array = []

            with concurrent.futures.ThreadPoolExecutor() as executor:
                node_urls = [
                    f"{server.base_url}{self.data_endpoint}/{node}?{self.single_node_query}"
                    for node in node_ids
                ]
                futures = [
                    executor.submit(self.make_api_call, node_url, server.auth_cookie)
                    for node_url in node_urls
                ]

            for future in concurrent.futures.as_completed(futures):
                try:
                    response_data = future.result()
                    node_array.append(response_data)
                except Exception as e:
                    print(
                        f"Exception occurred while processing individual node. Exception: {e}"
                    )
                    self.errors.append(e)

            print(f"{len(node_array)}: devices called")
            self.write_data(self.individual_target_dir, server.name, node_array)
            self.dbutils.fs.rm(file.path)

In [ ]:
# Declare and retrieve our input param that shows what device type to run this for.
dbutils.widgets.text("device_type", "unknown")
device_type = dbutils.widgets.get("device_type")

device_retrievers = {
    "routers": DataRetriever(
        [
            ServerInfo("ldxx90nms29", "http://ldxx90nms29.dx.deere.com"),
            ServerInfo("lvin2qnms1", "http://lvin2qnms1.jdnet.deere.com"),
            ServerInfo("lbne23nms2", "http://lbne23nms3.jdnet.deere.com"),
            ServerInfo("lbeinhnms2", "http://lbeinhnms2.jdnet.deere.com"),
            ServerInfo("lpunpunms2", "http://lpunpunms2.jdnet.deere.com"),
            ServerInfo("lman34nms2", "http://lman34nms2.jdnet.deere.com"),
            ServerInfo("ldxx90nms14", "http://ldxx90nms14.dx.deere.com"),
        ],
        "/mnt/edl/raw/nmis_dc_logs/routers/unprocessed",
        'query=["config.nodeType","router"]&properties=["info."]',
        dbutils,
    ),
    "switches": DataRetriever(
        [
            ServerInfo("lvin2qnms1", "http://lvin2qnms1.jdnet.deere.com"),
            ServerInfo("lbne23nms2", "http://lbne23nms3.jdnet.deere.com"),
            ServerInfo("lbeinhnms2", "http://lbeinhnms2.jdnet.deere.com"),
            ServerInfo("lpunpunms2", "http://lpunpunms2.jdnet.deere.com"),
        ],
        "/mnt/edl/raw/nmis_dc_logs/switches/unprocessed",
        'query=["config.nodeType","switch"]&properties=["info."]',
        dbutils,
    ),
    "switches_with_individual_nodes": DataRetriever(
        [
            ServerInfo("ldxx90nms14", "http://ldxx90nms14.dx.deere.com"),
            ServerInfo("lman34nms2", "http://lman34nms2.jdnet.deere.com"),
        ],
        "/mnt/edl/raw/nmis_dc_logs/switches/unprocessed/node_ids",
        'query=["config.nodeType","switch"]',
        dbutils,
        should_process_individual_node_ids=True,
        individual_target_dir="/mnt/edl/raw/nmis_dc_logs/switches/unprocessed",
    ),
}
device_retrievers[device_type].process_all_servers()